<h2>This is the code for training models. The running time exceeded 15 minutes, so I separated the training and inference. 
The inference code is here</h2>
<a href='https://www.kaggle.com/code/jiaoyouzhang/cmi-2025-only-lightgbm'>
  https://www.kaggle.com/code/jiaoyouzhang/cmi-2025-only-lightgbm
</a>

In [5]:
import numpy as np
import pandas as pd
import polars as pl
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import f1_score, classification_report
import lightgbm as lgb
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import joblib

# Import evaluation API
# import kaggle_evaluation.cmi_inference_server
pd.set_option('display.max_rows', 500)
random_seed = 42

In [6]:
train = pl.read_csv('../../0_data/train.csv')
train_demo = pl.read_csv("../../0_data/train_demographics.csv")

In [7]:
train = train.drop(["sequence_type","orientation","behavior","phase"])
data = train.join(train_demo,on="subject",how="left")

In [8]:
def feature_engineering(data:pl.DataFrame):
    demographic_cols = [
    "adult_child", "age", "sex", "handedness",
    "height_cm", "shoulder_to_wrist_cm", "elbow_to_wrist_cm"
    ]
    target_col = "gesture"
    
    # All numeric sensor columns (everything except id, demo, target)
    stat_cols = [
        c for c in data.columns
        if c not in demographic_cols + [target_col, "sequence_id", "row_id","sequence_counter","subject"]
    ]
    
    # Build aggregation expressions
    agg_exprs = []
    
    # full-stats bundle for sensor columns
    for c in stat_cols:
        agg_exprs += [
            pl.col(c).mean().alias(f"{c}_mean"),
            pl.col(c).std().alias(f"{c}_std"),
            pl.col(c).var().alias(f"{c}_var"),
            # pl.col(c).mode().list.first().alias(f"{c}_mode"),
            #pl.col(c).quantile(0.05).alias(f"{c}_q5"),
            pl.col(c).quantile(0.25).alias(f"{c}_q25"),
            pl.col(c).median().alias(f"{c}_q50"),
            pl.col(c).quantile(0.75).alias(f"{c}_q75"),
            #pl.col(c).quantile(0.95).alias(f"{c}_q95"),
            pl.col(c).max().alias(f"{c}_max"),
            pl.col(c).min().alias(f"{c}_min"),
        ]
    
    # first() for demographics and target
    agg_exprs += [
        pl.col(c).first().alias(c) for c in demographic_cols + [target_col]
    ]
    
    # Group-by and aggregate
    cleaned_data = (
        data
        .group_by("sequence_id", maintain_order=True)
        .agg(agg_exprs)
    )
    return cleaned_data

In [9]:
cleaned_data = feature_engineering(data)

In [10]:
print(data.shape)
print(cleaned_data.shape)


(574945, 344)
(8151, 2665)


In [11]:
data.head(3)

row_id,sequence_id,sequence_counter,subject,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,…,tof_5_v34,tof_5_v35,tof_5_v36,tof_5_v37,tof_5_v38,tof_5_v39,tof_5_v40,tof_5_v41,tof_5_v42,tof_5_v43,tof_5_v44,tof_5_v45,tof_5_v46,tof_5_v47,tof_5_v48,tof_5_v49,tof_5_v50,tof_5_v51,tof_5_v52,tof_5_v53,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64
"""SEQ_000007_000000""","""SEQ_000007""",0,"""SUBJ_059520""","""Cheek - pinch skin""",6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,28.943842,31.822186,29.553024,28.592863,28.310535,131.0,134.0,132.0,135.0,98.0,74.0,64.0,60.0,-1.0,-1.0,152.0,153.0,141.0,89.0,68.0,63.0,-1.0,-1.0,-1.0,-1.0,…,128.0,121.0,119.0,121.0,129.0,-1.0,113.0,124.0,122.0,131.0,-1.0,-1.0,-1.0,-1.0,120.0,127.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
"""SEQ_000007_000001""","""SEQ_000007""",1,"""SUBJ_059520""","""Cheek - pinch skin""",6.949219,6.214844,3.125,0.143494,-0.340271,-0.42865,-0.824524,29.340816,31.874645,29.79174,28.663383,28.406172,130.0,138.0,131.0,135.0,101.0,76.0,66.0,61.0,-1.0,-1.0,156.0,155.0,141.0,93.0,74.0,64.0,-1.0,-1.0,-1.0,-1.0,…,133.0,127.0,123.0,127.0,134.0,-1.0,116.0,122.0,123.0,126.0,-1.0,-1.0,-1.0,-1.0,122.0,129.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
"""SEQ_000007_000002""","""SEQ_000007""",2,"""SUBJ_059520""","""Cheek - pinch skin""",5.722656,5.410156,5.421875,0.219055,-0.274231,-0.356934,-0.865662,30.339359,30.935045,30.090014,28.796087,28.529778,137.0,136.0,147.0,109.0,90.0,81.0,74.0,74.0,-1.0,164.0,165.0,146.0,106.0,94.0,77.0,77.0,-1.0,-1.0,-1.0,180.0,…,-1.0,156.0,136.0,135.0,134.0,-1.0,133.0,142.0,131.0,130.0,132.0,136.0,-1.0,-1.0,112.0,121.0,123.0,125.0,-1.0,-1.0,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0


In [12]:
cleaned_data.head(3)

sequence_id,acc_x_mean,acc_x_std,acc_x_var,acc_x_q25,acc_x_q50,acc_x_q75,acc_x_max,acc_x_min,acc_y_mean,acc_y_std,acc_y_var,acc_y_q25,acc_y_q50,acc_y_q75,acc_y_max,acc_y_min,acc_z_mean,acc_z_std,acc_z_var,acc_z_q25,acc_z_q50,acc_z_q75,acc_z_max,acc_z_min,rot_w_mean,rot_w_std,rot_w_var,rot_w_q25,rot_w_q50,rot_w_q75,rot_w_max,rot_w_min,rot_x_mean,rot_x_std,rot_x_var,rot_x_q25,…,tof_5_v60_q25,tof_5_v60_q50,tof_5_v60_q75,tof_5_v60_max,tof_5_v60_min,tof_5_v61_mean,tof_5_v61_std,tof_5_v61_var,tof_5_v61_q25,tof_5_v61_q50,tof_5_v61_q75,tof_5_v61_max,tof_5_v61_min,tof_5_v62_mean,tof_5_v62_std,tof_5_v62_var,tof_5_v62_q25,tof_5_v62_q50,tof_5_v62_q75,tof_5_v62_max,tof_5_v62_min,tof_5_v63_mean,tof_5_v63_std,tof_5_v63_var,tof_5_v63_q25,tof_5_v63_q50,tof_5_v63_q75,tof_5_v63_max,tof_5_v63_min,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm,gesture
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,str
"""SEQ_000007""",6.153098,1.334155,1.779969,5.566406,6.488281,6.988281,9.015625,3.613281,3.91557,3.048287,9.292055,3.53125,5.488281,5.984375,6.519531,-2.019531,5.577782,2.337517,5.463986,3.660156,4.964844,6.875,9.792969,1.09375,0.263574,0.069033,0.004765,0.209961,0.254578,0.322571,0.379272,0.134399,-0.280817,0.056597,0.003203,-0.310242,…,-1.0,91.0,99.0,115.0,-1.0,51.877193,49.119777,2412.752506,-1.0,88.0,97.0,114.0,-1.0,55.947368,52.207492,2725.62218,-1.0,86.0,95.0,226.0,-1.0,41.192982,46.573843,2169.122807,-1.0,-1.0,92.0,99.0,-1.0,0,12,1,1,163.0,52,24.0,"""Cheek - pinch skin"""
"""SEQ_000008""",3.400506,1.087142,1.181878,2.574219,3.4375,4.4140625,5.90625,1.734375,5.311179,3.268073,10.6803,0.546875,7.0,7.785156,8.667969,-0.222656,6.581629,2.475402,6.127617,4.480469,5.839844,9.5703125,11.074219,1.722656,0.243493,0.064414,0.004149,0.196777,0.2265625,0.319092,0.34198,0.157593,-0.117145,0.049384,0.002439,-0.160339,…,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,24,1,1,173.0,49,26.0,"""Forehead - pull hairline"""
"""SEQ_000013""",-7.058962,1.295184,1.677501,-8.136719,-7.144531,-6.222656,-3.347656,-9.25,2.346182,2.564639,6.577376,2.15625,3.3828125,3.917969,4.683594,-3.273438,-6.068544,1.330784,1.770986,-6.617188,-5.851562,-5.277344,-3.515625,-10.945312,0.392208,0.150629,0.022689,0.404968,0.439514,0.48175,0.540771,0.061157,0.340804,0.182002,0.033125,0.22937,…,-1.0,-1.0,-1.0,-1.0,-1.0,3.603774,33.515978,1123.320755,-1.0,-1.0,-1.0,243.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,12,1,1,157.0,44,26.0,"""Cheek - pinch skin"""


In [6]:
target_col = "gesture"
pdf = cleaned_data.to_pandas()  # keeps nullable dtypes

le = LabelEncoder()
y = le.fit_transform(pdf[target_col])
X = pdf.drop(columns=[target_col, "sequence_id"])         # drop id + label

joblib.dump(le, 'le.joblib')

['le.joblib']

In [7]:
def competition_metric(y_true, y_pred, le_instance, all_original_gestures):
    """
    Competition metric calculation
    """
    bfrb_gestures = [g for g in all_original_gestures if g in le_instance.classes_]
    
    # Binary F1: All are Target in this filtered dataset
    y_true_binary = np.ones_like(y_true, dtype=int)
    y_pred_binary = np.ones_like(y_pred, dtype=int)
    binary_f1 = f1_score(y_true_binary, y_pred_binary, average='binary', pos_label=1, zero_division=0)
    
    # Macro F1: specific gesture classification
    macro_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    final_score = (binary_f1 + macro_f1) / 2
    return final_score, binary_f1, macro_f1

In [8]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []
models = []

all_original_gestures_in_train = pdf['gesture'].unique()

callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=100)]

# LightGBM model with cross-validation
print("\nTraining LightGBM models with cross-validation...")
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print(f"\nFold {fold + 1}/5")
    
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]
    
    # LightGBM model with GPU acceleration
    model = lgb.LGBMClassifier(
        objective='multiclass',
        n_estimators= 1000,
        learning_rate= 0.08,
        max_depth= 15,
        reg_alpha= 0.8,
        lambda_l2= 4.0,  
        num_leaves=31, 
        min_child_samples= 32,
        colsample_bytree= 0.85,
        subsample= 0.5,
        subsample_freq=0,
        cat_smooth=20.0,
        is_unbalance=True,
        max_bin=127,
        verbose=-1,  
        metric='multi_logloss',   
        device='gpu',  
    )
    
    # Train model with verbose output
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],  
        eval_metric='multi_logloss',  
        callbacks=callbacks
    )
    
    # Predict
    y_pred_fold = model.predict(X_val_fold)
    
    # Calculate score
    score, binary_f1, macro_f1 = competition_metric(
        y_val_fold, y_pred_fold, le, all_original_gestures_in_train
    )
    
    cv_scores.append(score)
    models.append(model)
    joblib.dump(model, f'model_lgb{fold}.joblib')
    
    print(f"Fold {fold + 1} - Competition Score: {score:.4f} (Binary F1: {binary_f1:.4f}, Macro F1: {macro_f1:.4f})")

    # feature importances
    feature_importances = model.feature_importances_    
    feature_names = X.columns    
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    })    
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)    
    print(feature_importance_df)

print(f"\nCross-validation results:")
print(f"Mean CV Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores) * 2:.4f})")
print(f"Individual fold scores: {cv_scores}")

# Train final model on all data with GPU acceleration
print("\nTraining final model on all training data...")


Training LightGBM models with cross-validation...

Fold 1/5


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[253]	valid_0's multi_logloss: 1.04424
Fold 1 - Competition Score: 0.8104 (Binary F1: 1.0000, Macro F1: 0.6208)
            Feature  Importance
23        acc_z_min        1055
14        acc_y_max         949
7         acc_x_min         812
13        acc_y_q75         743
15        acc_y_min         684
...             ...         ...
1031  tof_2_v52_min           0
1039  tof_2_v53_min           0
1055  tof_2_v55_min           0
655    tof_2_v5_min           0
967   tof_2_v44_min           0

[2663 rows x 2 columns]

Fold 2/5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[229]	valid_0's multi_logloss: 1.00521
Fold 2 - Competition Score: 0.8109 (Binary F1: 1.0000, Macro F1: 0.6218)
            Feature  Importance
23        acc_z_min        1095
14        acc_y_max         893
7         acc_x_min         778
13        acc_y_q75         683
46 